### Загружаем данные из csv файлов

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
import seaborn as sns

%matplotlib inline

In [2]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [3]:
df.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644


In [4]:
df.shape

(10000, 20)

In [5]:
df.dtypes

Id                 int64
DistrictId         int64
Rooms            float64
Square           float64
LifeSquare       float64
KitchenSquare    float64
Floor              int64
HouseFloor       float64
HouseYear          int64
Ecology_1        float64
Ecology_2         object
Ecology_3         object
Social_1           int64
Social_2           int64
Social_3           int64
Healthcare_1     float64
Helthcare_2        int64
Shops_1            int64
Shops_2           object
Price            float64
dtype: object

In [6]:
df.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [7]:
df.nunique()

Id               10000
DistrictId         205
Rooms                9
Square           10000
LifeSquare        7887
KitchenSquare       58
Floor               33
HouseFloor          44
HouseYear           97
Ecology_1          129
Ecology_2            2
Ecology_3            2
Social_1            51
Social_2           142
Social_3            30
Healthcare_1        79
Helthcare_2          7
Shops_1             16
Shops_2              2
Price            10000
dtype: int64

In [8]:
df_test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,2623.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.910000,56.449500,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1146.657263,1.31940,4.242800
std,4832.674037,44.179466,0.838594,19.092787,17.825287,9.950018,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1044.744231,1.47994,4.777365
min,1.000000,0.000000,0.000000,1.378543,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,325.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,900.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,1548.000000,2.00000,6.000000
max,16795.000000,212.000000,17.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


### Обработка данных, заполнение пробелов и выбросов

In [9]:
df.replace({'Ecology_2': {'A':0, 'B':1} ,'Ecology_3': {'A':0, 'B':1}, 'Shops_2': {'A':0, 'B':1}}, inplace = True)

In [10]:
df_test.replace({'Ecology_2': {'A':0, 'B':1} ,'Ecology_3': {'A':0, 'B':1}, 'Shops_2': {'A':0, 'B':1}}, inplace = True)

In [11]:
df['HouseYear'].value_counts()

1977        2652
2017         308
2016         305
2015         299
2014         210
1970         194
1969         190
1972         178
1971         177
2018         175
1974         169
1968         162
1965         161
1963         158
1976         155
1973         150
1964         150
1975         150
1967         146
1966         140
1978         129
1962         125
1960         121
1961         113
1982         111
1981         108
1979         107
1980         107
2005         102
1983         101
            ... 
1952          10
1953          10
1931           7
1935           7
1950           7
1939           6
1918           5
1933           5
1937           5
1934           4
1938           4
1951           4
1940           4
1930           3
1942           3
1936           3
1919           3
1917           3
1912           2
1929           2
1932           2
1948           2
1941           2
4968           1
1914           1
1947           1
1916           1
1910          

In [12]:
df.loc[df["HouseYear"] == 4968, "HouseYear"]=1968
df.loc[df["HouseYear"] == 20052011, "HouseYear"]=2011

In [13]:
df.loc[df["HouseYear"] >2020, "HouseYear"]

Series([], Name: HouseYear, dtype: int64)

In [14]:
df_test.loc[df_test["HouseYear"] >2020, "HouseYear"]

Series([], Name: HouseYear, dtype: int64)

### Обработка этажей

In [15]:
df.query('Floor>HouseFloor').count()

Id               1825
DistrictId       1825
Rooms            1825
Square           1825
LifeSquare       1210
KitchenSquare    1825
Floor            1825
HouseFloor       1825
HouseYear        1825
Ecology_1        1825
Ecology_2        1825
Ecology_3        1825
Social_1         1825
Social_2         1825
Social_3         1825
Healthcare_1      861
Helthcare_2      1825
Shops_1          1825
Shops_2          1825
Price            1825
dtype: int64

In [16]:
df.loc[df['Floor']>df['HouseFloor'],'HouseFloor'] =df.loc[df['Floor']>df['HouseFloor'],'Floor']

In [17]:
df_test.query('Floor>HouseFloor').count()

Id               884
DistrictId       884
Rooms            884
Square           884
LifeSquare       575
KitchenSquare    884
Floor            884
HouseFloor       884
HouseYear        884
Ecology_1        884
Ecology_2        884
Ecology_3        884
Social_1         884
Social_2         884
Social_3         884
Healthcare_1     431
Helthcare_2      884
Shops_1          884
Shops_2          884
dtype: int64

In [18]:
df_test.loc[df_test['Floor']>df_test['HouseFloor'],'HouseFloor'] =df_test.loc[df_test['Floor']>df_test['HouseFloor'],'Floor']

### Обработка выбросов по Rooms

In [19]:
df['Rooms'].value_counts()

2.0     3880
1.0     3705
3.0     2235
4.0      150
5.0       18
0.0        8
10.0       2
19.0       1
6.0        1
Name: Rooms, dtype: int64

In [20]:
df_test['Rooms'].value_counts()

2.0     2030
1.0     1769
3.0     1099
4.0       90
5.0        7
0.0        2
6.0        2
17.0       1
Name: Rooms, dtype: int64

In [21]:
df.query('(Rooms == 0)|(Rooms>5)')

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
377,5927,57,10.0,59.056975,36.223072,10.0,22,22.0,2002,0.090799,1,1,74,19083,2,NaN,5,15,1,317265.323792
1397,12638,27,0.0,138.427694,136.215499,0.0,4,4.0,2016,0.075424,1,1,11,3097,0,NaN,0,0,1,268394.744389
1454,8491,1,19.0,42.006046,21.779288,7.0,17,17.0,2014,0.007122,1,1,1,264,0,NaN,0,1,1,78364.616704
1981,7917,27,0.0,212.932361,211.231125,0.0,2,3.0,2008,0.211401,1,1,9,1892,0,NaN,0,1,1,302211.260887
2170,14003,99,6.0,59.414334,38.702244,6.0,7,9.0,1969,0.033494,1,1,66,10573,1,1322.0,3,8,1,229661.964416
2269,7317,27,0.0,41.790881,NaN,0.0,13,13.0,1977,0.211401,1,1,9,1892,0,NaN,0,1,1,98129.976788
3911,770,28,0.0,49.483501,NaN,0.0,16,16.0,2015,0.118537,1,1,30,6207,1,1183.0,1,0,1,217009.338463
4366,456,6,0.0,81.491446,NaN,0.0,4,4.0,1977,0.243205,1,1,5,1564,0,540.0,0,0,1,212864.799112
4853,3224,27,0.0,2.377248,0.873147,0.0,1,1.0,1977,0.017647,1,1,2,469,0,NaN,0,0,1,126596.941798
6149,3159,88,0.0,38.697117,19.345131,9.0,9,16.0,1982,0.127376,1,1,43,8429,3,NaN,3,9,1,158998.110646


Замена аномалий соотношением Square/Rooms.

In [22]:
def room_fill(X):
    avg_r = X['Square'].mean()/X['Rooms'].mean()
    X.loc[((X['Rooms']==0)&(X['Square']>=50))|(X['Rooms']>5),'Rooms'] = round(X.loc[((X['Rooms']==0)&(X['Square']>=50))|(X['Rooms']>5),'Square']/avg_r)
    X.loc[X['Rooms']>5,'Rooms'] = 5
    return

In [23]:
room_fill(df)

In [24]:
df.query('(Rooms == 0)|(Rooms>5)')

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
2269,7317,27,0.0,41.790881,NaN,0.0,13,13.0,1977,0.211401,1,1,9,1892,0,NaN,0,1,1,98129.976788
3911,770,28,0.0,49.483501,NaN,0.0,16,16.0,2015,0.118537,1,1,30,6207,1,1183.0,1,0,1,217009.338463
4853,3224,27,0.0,2.377248,0.873147,0.0,1,1.0,1977,0.017647,1,1,2,469,0,NaN,0,0,1,126596.941798
6149,3159,88,0.0,38.697117,19.345131,9.0,9,16.0,1982,0.127376,1,1,43,8429,3,NaN,3,9,1,158998.110646


In [25]:
room_fill(df_test)

In [26]:
df_test.query('(Rooms == 0)|(Rooms>5)')

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2


### Обработка площади кухни

In [27]:
df['KitchenSquare'].value_counts()

1.0       2460
8.0       1306
5.0       1169
10.0      1075
6.0       1038
9.0        843
0.0        697
7.0        609
12.0       249
11.0       233
13.0        67
14.0        51
4.0         39
15.0        31
3.0         22
16.0        16
20.0        14
17.0        12
19.0        11
18.0         6
2.0          4
22.0         3
41.0         2
112.0        2
25.0         2
43.0         2
51.0         2
37.0         2
32.0         2
30.0         2
58.0         2
72.0         1
96.0         1
66.0         1
48.0         1
40.0         1
2014.0       1
35.0         1
60.0         1
78.0         1
27.0         1
84.0         1
62.0         1
42.0         1
63.0         1
39.0         1
1970.0       1
36.0         1
75.0         1
26.0         1
21.0         1
29.0         1
23.0         1
73.0         1
123.0        1
31.0         1
53.0         1
54.0         1
Name: KitchenSquare, dtype: int64

In [28]:
def kitchen_fill(X):
    for i in range(0,6):
        X.loc[(X['Rooms'] == i)&((X['KitchenSquare']>29)&(X['KitchenSquare']>X['Square']*0.50 )),'KitchenSquare']= X.loc[X['Rooms'] == i, 'KitchenSquare'].median()
    return    

In [29]:
kitchen_fill(df)

In [30]:
df.query('KitchenSquare>29')

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1867,3267,58,3.0,138.980817,138.004600,43.0,2,2.0,1977,0.437885,1,1,23,5735,3,1084.0,0,5,1,169528.896664
3368,6674,54,1.0,168.086347,125.486563,30.0,18,28.0,2011,0.006076,1,1,30,5285,0,645.0,6,6,1,367353.574566
3431,7111,27,2.0,139.070296,89.787666,42.0,5,5.0,2016,0.014073,1,1,2,475,0,NaN,0,0,1,153946.777452
5087,15947,129,4.0,185.906396,84.830074,31.0,5,5.0,2015,0.020741,1,1,24,5613,5,1340.0,2,5,1,340273.238253
5494,300,31,2.0,106.427686,34.230415,32.0,4,40.0,2013,0.151346,1,0,32,5889,10,830.0,4,1,1,624549.354546
6988,8685,62,5.0,124.933289,81.122369,30.0,5,5.0,2017,0.072158,1,1,2,629,1,NaN,0,0,0,271919.866048
7201,3233,129,4.0,186.692602,87.487080,32.0,3,3.0,2011,0.020741,1,1,24,5613,5,1340.0,2,5,1,423443.464367


In [31]:
kitchen_fill(df_test)

In [32]:
df_test.query('KitchenSquare>29')

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
625,2249,62,1.0,77.157675,78.281763,34.0,5,24.0,2016,0.072158,1,1,2,629,1,NaN,0,0,0
803,3747,23,5.0,139.045763,89.790274,42.0,1,3.0,2015,0.014073,1,1,2,475,0,NaN,0,0,1
870,13736,27,1.0,141.540215,91.704806,42.0,3,3.0,2013,0.014073,1,1,2,475,0,NaN,0,0,1
1447,1968,27,2.0,138.039182,92.484098,42.0,3,3.0,2016,0.014073,1,1,2,475,0,NaN,0,0,1
4546,5254,186,2.0,88.618122,25.457811,30.0,5,19.0,1985,0.007659,1,1,35,7077,56,2301.0,3,19,1


### Обработка Square

In [33]:
df.loc[df['Square']>200]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1981,7917,27,5.0,212.932361,211.231125,0.0,2,3.0,2008,0.211401,1,1,9,1892,0,NaN,0,1,1,302211.260887
1982,5548,86,5.0,275.645284,233.949309,26.0,12,37.0,2011,0.161976,1,0,31,7010,5,4508.0,3,7,1,455264.882666
4262,28,9,2.0,604.705972,NaN,1.0,17,18.0,1977,0.161532,1,1,25,5648,1,30.0,2,4,1,187717.242538
4690,2307,102,1.0,409.425181,410.639749,10.0,4,4.0,2016,0.238617,1,1,26,3889,6,705.0,3,6,1,90470.430830
6977,11602,30,2.0,641.065193,638.163193,10.0,20,20.0,2019,0.000078,1,1,22,6398,141,1046.0,3,23,1,133529.681562
9910,16568,27,4.0,200.334539,201.627361,25.0,1,2.0,2013,0.041116,1,1,53,14892,4,NaN,1,4,1,528560.506016


In [34]:
df_test.loc[df_test['Square']>200]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
3217,4058,27,5.0,223.453689,104.113552,16.0,2,2.0,2017,0.041116,1,1,53,14892,4,NaN,1,4,1
3909,1397,76,3.0,221.138768,118.055342,4.0,32,48.0,2011,0.000000,1,1,7,1660,39,1786.0,1,5,1


In [35]:
df.loc[(df['Square']>200)&(df['HouseYear']<2000),'Square'] = df.loc[(df['Rooms']==2)&(df['HouseYear']<2000),'Square'].mean()

In [36]:
df.query('(Square<KitchenSquare)|(Square<=13)')

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
212,1748,88,2.0,5.497061,67.628717,1.0,24,24.0,1977,0.127376,1,1,43,8429,3,NaN,3,9,1,412511.088764
1316,11526,27,1.0,4.633498,1.969969,1.0,18,18.0,1977,0.011654,1,1,4,915,0,NaN,0,0,1,107604.269441
1608,10202,6,1.0,2.596351,4.604943,1.0,3,25.0,2014,0.243205,1,1,5,1564,0,540.0,0,0,1,137597.601458
3280,10527,27,1.0,4.380726,40.805837,1.0,10,17.0,2013,0.211401,1,1,9,1892,0,NaN,0,1,1,97560.720383
3413,9487,5,1.0,5.129222,5.549458,1.0,1,1.0,1977,0.150818,1,1,16,3433,4,2643.0,4,5,1,369472.403061
4853,3224,27,0.0,2.377248,0.873147,0.0,1,1.0,1977,0.017647,1,1,2,469,0,NaN,0,0,1,126596.941798
4900,4504,27,3.0,4.390331,5.610772,1.0,8,19.0,2016,0.211401,1,1,9,1892,0,NaN,0,1,1,161379.067034
6392,14786,1,1.0,1.136859,4.525736,1.0,3,3.0,1977,0.007122,1,1,1,264,0,NaN,0,1,1,181434.825589
8030,13265,1,3.0,4.823679,79.767964,0.0,6,17.0,1977,0.007122,1,1,1,264,0,NaN,0,1,1,237716.681261
8283,15744,34,1.0,1.988943,2.642219,1.0,21,21.0,1977,0.069753,1,1,53,13670,4,NaN,1,11,1,458378.777006


In [37]:
def square_fill(X):
    for i in range(0,6):
        X.loc[(X['Rooms'] == i)&((X['Square']<13)|(X['Square']<X['KitchenSquare'])),'Square']= X.loc[X['Rooms'] == i, 'Square'].mean()
    return

In [38]:
square_fill(df)
square_fill(df_test)

In [39]:
df.query('(Square<KitchenSquare)|(Square<=13)')

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price


In [40]:
df_test.query('(Square<KitchenSquare)|(Square<=13)')

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2


In [41]:
df.loc[df['Square'].isnull()]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price


In [42]:
LifeSquareRate = df.loc[~((df['LifeSquare'].isnull())|(df['LifeSquare']>df['Square'])|(df['LifeSquare']>1000)), 'LifeSquare'].mean() \
/df.loc[~((df['LifeSquare'].isnull())|(df['LifeSquare']>df['Square'])|(df['LifeSquare']>1000)), 'Square'].mean()

In [43]:
# заполняем LifeSquare как Square * LifeSquareRate
def fill_LifeSquare(X):
    X.loc[(X['Square'] < X['LifeSquare']) | (X['LifeSquare'].isnull())|(X['LifeSquare']>1000), 'LifeSquare'] = X.loc[(X['Square'] < X['LifeSquare']) | (X['LifeSquare'].isnull())|(X['LifeSquare']>1000), 'Square']*LifeSquareRate
    return

In [44]:
fill_LifeSquare(df)

In [45]:
df.loc[(df['LifeSquare'].isnull())|(df['LifeSquare']>df['Square'])|(df['LifeSquare']>1000)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price


In [46]:
fill_LifeSquare(df_test)

In [47]:
df_test.loc[(df['LifeSquare'].isnull())|(df['LifeSquare']>df['Square'])|(df['LifeSquare']>1000)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2


In [48]:
from sklearn.ensemble import RandomForestRegressor
lsmodel = RandomForestRegressor(n_estimators=1000, max_depth=12, random_state=19)

In [49]:
df.head(15)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,1,1,33,7976,5,NaN,0,11,1,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,1,1,46,10309,1,240.0,1,16,1,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,1,1,34,7759,0,229.0,1,3,1,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,1,1,23,5735,3,1084.0,0,5,1,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,1,1,35,5776,1,2078.0,2,4,1,150226.531644
5,12915,59,3.0,80.384479,46.683720,12.0,5,17.0,2011,0.309479,1,1,35,7715,4,990.0,0,6,1,215898.447742
6,14549,154,2.0,62.254114,37.160377,7.0,3,5.0,1960,0.460556,1,1,20,4386,14,NaN,1,5,1,296021.204377
7,11993,74,2.0,80.312926,50.438467,0.0,14,14.0,1977,0.075779,1,1,6,1437,3,NaN,0,2,1,221244.156664
8,5172,1,2.0,64.511437,40.514749,1.0,9,17.0,1977,0.007122,1,1,1,264,0,NaN,0,1,1,229102.795999
9,8649,23,1.0,46.461409,18.915552,8.0,13,17.0,2014,0.075779,1,1,6,1437,3,NaN,0,2,1,95380.220993


### Строим базовую модель

In [50]:
X_train, X_test,y_train,y_test = train_test_split(df.drop(['Id','Healthcare_1', 'Price'], axis = 'columns'), 
                                                  df['Price'], test_size = 0.2, random_state=33)

In [51]:
# Модель без Id и Healthcare_1
model_RFR = RandomForestRegressor(max_depth=15, n_estimators=200, random_state=55)
model_RFR.fit(X_train,y_train)
first_predict = model_RFR.predict(X_test)
r2_score(y_test, first_predict)

0.72239014910373078

In [52]:
# переобучаем на всех данных
model_RFR = RandomForestRegressor(max_depth=15, n_estimators=200, random_state=55)
model_RFR.fit(df.drop(['Id','Healthcare_1', 'Price'], axis = 'columns'), df['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=False, random_state=55,
           verbose=0, warm_start=False)

In [53]:
# важность признаков
importance = pd.DataFrame()
importance['name'] = df.drop(['Id','Healthcare_1', 'Price'], axis = 'columns').columns
importance['values'] = model_RFR.feature_importances_
importance.sort_values('values', ascending = False)

,name,values
2,Square,0.395181
11,Social_1,0.100697
12,Social_2,0.099629
1,Rooms,0.080927
13,Social_3,0.061124
0,DistrictId,0.049189
8,Ecology_1,0.043461
7,HouseYear,0.034439
3,LifeSquare,0.031222
6,HouseFloor,0.028730


In [54]:
first_predict = model_RFR.predict(df_test.drop(['Id','Healthcare_1'], axis = 'columns'))

In [55]:
df_test['Price'] = first_predict

In [56]:
df_test[['Id', 'Price']].to_csv('first_predict.csv', index = False)

### Заполнение HealthCare_1 RandomForestClassifier не улулчшило модель, наоборот, снизило скор

In [57]:
# Создадим новый признак Price_mdistrict - цена за квадратный метр
def price_dist(X):
    district = X['DistrictId'].unique()
    Price_mdistrict = {}
    for d in district:
        Price_mdistrict[d] = X.loc[X['DistrictId']==d, 'Price'].sum()/X.loc[X['DistrictId']==d, 'Square'].sum()
    X['Price_mdistrict'] = X['DistrictId'].map(Price_mdistrict)
    aver_per_m = X.loc[X['DistrictId'], 'Price'].sum()/X.loc[X['DistrictId'], 'Square'].sum()
    return  Price_mdistrict, aver_per_m

In [58]:
df.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,1,1,33,7976,5,NaN,0,11,1,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,1,1,46,10309,1,240.0,1,16,1,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,1,1,34,7759,0,229.0,1,3,1,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,1,1,23,5735,3,1084.0,0,5,1,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,1,1,35,5776,1,2078.0,2,4,1,150226.531644


In [59]:
Price_mdistrict, aver_per_m = price_dist(df)

In [60]:
X_train, X_test,y_train,y_test = train_test_split(df.drop(['Id','Healthcare_1', 'Price'], axis = 'columns'), 
                                                  df['Price'], test_size = 0.2, random_state=33)

In [61]:
# Модель c Price_mdistrict 
model_RFR = RandomForestRegressor(max_depth=16, n_estimators=300, random_state=55)
model_RFR.fit(X_train,y_train)
second_predict = model_RFR.predict(X_test)
r2_score(y_test, second_predict)

0.75714635599727498

In [62]:
def price_dist_test(X):
    district = X['DistrictId'].unique()
    Price_mtest = {}
    for d in district:
        if d in Price_mdistrict.keys():
            Price_mtest[d] = Price_mdistrict[d]
        else:
            Price_mtest[d] = aver_per_m
    X['Price_mdistrict'] = X['DistrictId'].map(Price_mtest)
    return 

In [63]:
price_dist_test(df_test)

In [64]:
df_test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,Price_mdistrict
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,...,1,11,2748,1,NaN,0,0,1,164459.364589,2957.990008
1,15856,74,2.0,69.263183,43.498961,1.0,6,6.0,1977,0.075779,...,1,6,1437,3,NaN,0,2,1,222294.166907,3272.322491
2,5480,190,1.0,13.597819,8.539760,12.0,2,5.0,1909,0.000000,...,1,30,7538,87,4702.0,5,5,1,199080.437480,7735.224377
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,...,1,23,4583,3,NaN,3,3,1,345222.331424,4063.439258
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,...,1,2,629,1,NaN,0,0,0,143496.226162,2705.917546


In [65]:
# переобучаем на всех данных
model_RFR = RandomForestRegressor(max_depth=16, n_estimators=300, random_state=55)
model_RFR.fit(df.drop(['Id','Healthcare_1', 'Price'], axis = 'columns'), df['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=16,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=300, n_jobs=1, oob_score=False, random_state=55,
           verbose=0, warm_start=False)

In [66]:
third_predict = model_RFR.predict(df_test.drop(['Id','Healthcare_1','Price'], axis = 'columns'))

In [67]:
df_test['Price'] = third_predict

In [68]:
df_test[['Id', 'Price']].to_csv('third_predict.csv', index = False)

### Удалим малозначимые признаки

In [69]:
X_train, X_test,y_train,y_test = train_test_split(df.drop(['Id','Healthcare_1', 'Helthcare_2', 'Shops_2', 'Ecology_2', 'Ecology_3', 'Price'], axis = 'columns'), 
                                                  df['Price'], test_size = 0.2, random_state=33)

In [73]:
model_RFR = RandomForestRegressor(max_depth=16, n_estimators=300, random_state=55)
model_RFR.fit(X_train,y_train)
second_predict = model_RFR.predict(X_test)
r2_score(y_test, second_predict)

0.75767178911603728

In [79]:
model_RFR = RandomForestRegressor(max_depth=16, n_estimators=300, random_state=55)
model_RFR.fit(df.drop(['Id','Healthcare_1', 'Helthcare_2', 'Shops_2', 'Ecology_2', 'Ecology_3', 'Price'], axis = 'columns'), df['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=16,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=300, n_jobs=1, oob_score=False, random_state=55,
           verbose=0, warm_start=False)

In [80]:
fourth_predict = model_RFR.predict(df_test.drop(['Id','Healthcare_1', 'Helthcare_2', 'Shops_2', 'Ecology_2', 'Ecology_3', 'Price'], axis = 'columns'))

In [81]:
df_test['Price'] = fourth_predict

In [82]:
df_test[['Id', 'Price']].to_csv('fourth_predict.csv', index = False)

### Без средней цены за квадрат

In [83]:
model_RFR = RandomForestRegressor(max_depth=15, n_estimators=200, random_state=55)
model_RFR.fit(df.drop(['Id','Healthcare_1', 'Helthcare_2', 'Shops_2', 'Ecology_2', 'Ecology_3', 'Price','Price_mdistrict'], axis = 'columns'), df['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=False, random_state=55,
           verbose=0, warm_start=False)

In [84]:
fifth_predict = model_RFR.predict(df_test.drop(['Id','Healthcare_1', 'Helthcare_2', 'Shops_2', 'Ecology_2', 'Ecology_3', 'Price','Price_mdistrict'], axis = 'columns'))

In [85]:
df_test['Price'] = fifth_predict
df_test[['Id', 'Price']].to_csv('fifth_predict.csv', index = False)

### Вывод: найлучший результат на Kaggle fourth_predict, без слабых признаков и со средней ценой за квадрат. скор на Kaggle 0.7422